In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tqdm
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
import requests
from nba_api.stats.static.players import find_players_by_full_name
from adjustText import adjust_text
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import time
from PIL import Image
from collections import defaultdict

In [2]:
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2

from nba_api.stats.endpoints.playbyplay import PlayByPlay
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.boxscoretraditionalv2 import BoxScoreTraditionalV2
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.leaguedashteamshotlocations import LeagueDashTeamShotLocations
from nba_api.stats.endpoints.leaguedashplayerptshot import LeagueDashPlayerPtShot

from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats
from nba_api.stats.endpoints.leaguedashplayerstats import LeagueDashPlayerStats

from nba_api.stats.endpoints.leaguehustlestatsplayer import LeagueHustleStatsPlayer

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import tqdm
from time import sleep
import time
from PIL import Image
from io import BytesIO
from adjustText import adjust_text
from scipy.interpolate import interp1d

from nba_api.stats.static.players import find_players_by_full_name
from nba_api.stats.static.teams import find_teams_by_full_name, get_teams, find_team_name_by_id
from nba_api.stats.endpoints.playbyplayv2 import PlayByPlayV2
from nba_api.stats.endpoints.leaguegamefinder import LeagueGameFinder
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail

from nba_api.stats.endpoints.synergyplaytypes import SynergyPlayTypes
from nba_api.stats.endpoints.leaguedashptstats import LeagueDashPtStats


In [4]:
bucks_id = find_teams_by_full_name("Milwaukee Bucks")[0]['id']

In [5]:
bucks_df = ShotChartDetail(team_id=bucks_id, player_id=0, season_type_all_star='Playoffs', context_measure_simple='FGA', season_nullable='2020-21').get_data_frames()[0]

In [9]:
nets_game = bucks_df.loc[bucks_df.GAME_ID == '0042000211']

In [12]:
nets_game.ACTION_TYPE.unique()

array(['Jump Shot', 'Floating Jump shot', 'Dunk Shot', 'Pullup Jump shot',
       'Running Pull-Up Jump Shot', 'Driving Floating Jump Shot',
       'Layup Shot', 'Alley Oop Dunk Shot', 'Running Layup Shot',
       'Hook Shot', 'Putback Layup Shot', 'Tip Layup Shot',
       'Step Back Jump shot', 'Turnaround Hook Shot',
       'Driving Layup Shot', 'Running Jump Shot', 'Cutting Dunk Shot',
       'Turnaround Jump Shot', 'Driving Finger Roll Layup Shot',
       'Cutting Layup Shot', 'Tip Dunk Shot', 'Turnaround Fadeaway shot',
       'Turnaround Fadeaway Bank Jump Shot', 'Driving Reverse Dunk Shot',
       'Driving Floating Bank Jump Shot', 'Finger Roll Layup Shot',
       'Fadeaway Jump Shot', 'Reverse Layup Shot'], dtype=object)

In [13]:
created_shots = [
    'Turnaround Fadeaway Bank Jump Shot',
    'Pullup Jump shot',  'Fadeaway Jump Shot',
    'Step Back Jump shot', 'Driving Floating Bank Jump Shot',
    'Driving Floating Jump Shot',
    'Running Pull-Up Jump Shot', 'Turnaround Jump Shot',
]

In [29]:
pullups = [
    'Pullup Jump shot',
    'Running Pull-Up Jump Shot',
    'Step Back Jump shot',
]

In [30]:
df_create = nets_game.loc[nets_game.ACTION_TYPE.isin(pullups)]

In [31]:
all_create = bucks_df.loc[bucks_df.ACTION_TYPE.isin(pullups)]

In [32]:

df_create.groupby(('GAME_ID', 'SHOT_TYPE')).agg(
    {
        'SHOT_ATTEMPTED_FLAG': 'sum',
        'SHOT_MADE_FLAG': 'sum'
    }
).reset_index()

c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


,GAME_ID,SHOT_TYPE,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,0042000211,2PT Field Goal,6,4
1,0042000211,3PT Field Goal,14,4


In [33]:

grouped_all = all_create.groupby(('GAME_ID', 'SHOT_TYPE')).agg(
    {
        'SHOT_ATTEMPTED_FLAG': 'sum',
        'SHOT_MADE_FLAG': 'sum'
    }
).reset_index()

c:\users\danie\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [34]:
grouped_all.loc[:, 'PCT'] = grouped_all.SHOT_MADE_FLAG / (grouped_all.SHOT_ATTEMPTED_FLAG) * 100

In [28]:
df_create.loc[df_create.SHOT_MADE_FLAG == 1]

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
384,Shot Chart Detail,0042000211,31,201950,Jrue Holiday,1610612749,Milwaukee Bucks,1,9,31,...,Right Side Center(RC),24+ ft.,25,96,235,1,1,20210605,BKN,MIL
410,Shot Chart Detail,0042000211,156,201950,Jrue Holiday,1610612749,Milwaukee Bucks,1,1,23,...,Center(C),Less Than 8 ft.,7,3,71,1,1,20210605,BKN,MIL
420,Shot Chart Detail,0042000211,252,201950,Jrue Holiday,1610612749,Milwaukee Bucks,2,7,27,...,Center(C),Less Than 8 ft.,4,48,6,1,1,20210605,BKN,MIL
423,Shot Chart Detail,0042000211,273,201950,Jrue Holiday,1610612749,Milwaukee Bucks,2,6,11,...,Center(C),Less Than 8 ft.,4,26,39,1,1,20210605,BKN,MIL
428,Shot Chart Detail,0042000211,314,201950,Jrue Holiday,1610612749,Milwaukee Bucks,2,3,13,...,Center(C),24+ ft.,25,-1,258,1,1,20210605,BKN,MIL
430,Shot Chart Detail,0042000211,329,203114,Khris Middleton,1610612749,Milwaukee Bucks,2,2,14,...,Center(C),8-16 ft.,14,-70,132,1,1,20210605,BKN,MIL
432,Shot Chart Detail,0042000211,337,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,2,1,22,...,Center(C),24+ ft.,26,-19,262,1,1,20210605,BKN,MIL
444,Shot Chart Detail,0042000211,412,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,3,7,51,...,Center(C),Less Than 8 ft.,7,-2,79,1,1,20210605,BKN,MIL
446,Shot Chart Detail,0042000211,427,203114,Khris Middleton,1610612749,Milwaukee Bucks,3,6,6,...,Center(C),Less Than 8 ft.,2,-6,27,1,1,20210605,BKN,MIL
453,Shot Chart Detail,0042000211,479,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,3,3,10,...,Center(C),8-16 ft.,10,13,105,1,1,20210605,BKN,MIL


In [35]:
grouped_all

,GAME_ID,SHOT_TYPE,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,PCT
0,0042000121,2PT Field Goal,10,6,60.000000
1,0042000121,3PT Field Goal,5,1,20.000000
2,0042000122,2PT Field Goal,4,1,25.000000
3,0042000122,3PT Field Goal,12,4,33.333333
4,0042000123,2PT Field Goal,7,4,57.142857
5,0042000123,3PT Field Goal,7,2,28.571429
6,0042000124,2PT Field Goal,6,3,50.000000
7,0042000124,3PT Field Goal,8,0,0.000000
8,0042000211,2PT Field Goal,6,4,66.666667
9,0042000211,3PT Field Goal,14,4,28.571429


In [37]:
bucks_df.iloc[0]

GRID_TYPE                 Shot Chart Detail
GAME_ID                          0042000121
GAME_EVENT_ID                            15
PLAYER_ID                            201950
PLAYER_NAME                    Jrue Holiday
TEAM_ID                          1610612749
TEAM_NAME                   Milwaukee Bucks
PERIOD                                    1
MINUTES_REMAINING                        11
SECONDS_REMAINING                         8
EVENT_TYPE                      Missed Shot
ACTION_TYPE                       Jump Shot
SHOT_TYPE                    3PT Field Goal
SHOT_ZONE_BASIC           Above the Break 3
SHOT_ZONE_AREA         Left Side Center(LC)
SHOT_ZONE_RANGE                     24+ ft.
SHOT_DISTANCE                            25
LOC_X                                  -210
LOC_Y                                   138
SHOT_ATTEMPTED_FLAG                       1
SHOT_MADE_FLAG                            0
GAME_DATE                          20210522
HTM                             